In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv("../input/train.csv")

In [ ]:
train_data.head()

In [ ]:
import string

In [ ]:
question_text = train_data['question_text'].str.replace('[^\w\s]','')

In [ ]:
question_text = question_text.str.split(' ')

In [ ]:
question_text.head()

In [ ]:
q_list = []
for i,x in enumerate(question_text[:]):
   if isinstance(x,list): 
      if len(x)!=0:
        for y in x:
            q_list.append(y)

In [ ]:
q_list

In [ ]:
from collections import Counter
counts = Counter(q_list)
q_vocab = sorted(counts, key=counts.get, reverse=True)
q_vocab_to_int = {word: ii for ii, word in enumerate(q_vocab, 1)}

In [ ]:
q_int=[]
for q in question_text:
  q_int.append([q_vocab_to_int[word] for word in q])

In [ ]:
#q_int

In [ ]:
def pad_features(title_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(title_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(title_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [ ]:
seq_length = 25

q_features = pad_features(q_int, seq_length=seq_length)

## test statements - do not change - ##
assert len(q_features)==len(q_int), "Your features should have as many rows as reviews."
assert len(q_features[0])==seq_length, "Each feature row should contain seq_length values."
print(q_features[:30,:25])

In [ ]:
q_labels = train_data['target']

In [ ]:
#q_labels = q_labels.tolist()
q_labels = np.array(q_labels)

In [ ]:
split_frac = 0.6

## split data into training, validation, and test data (features and labels, x and y)
batch_size = 20
split_idx = int(len(q_features)*0.9)
split_idx = split_idx - (split_idx%batch_size)
train_x, val_x = q_features[:split_idx], q_features[split_idx+2:]
train_y, val_y = q_labels[:split_idx], q_labels[split_idx+2:]

#test_idx = int(len(remaining_x)*0.2)
#test_idx = test_idx - (test_idx%batch_size)
#val_x, test_x = remaining_x[:test_idx], remaining_x[:-test_idx+1]
#val_y, test_y = remaining_y[:test_idx], remaining_y[:-test_idx+1]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nTrain_y: \t\t{}".format(train_y.shape),
      "\nValidation set: \t{}".format(val_x.shape))
      #"\nTest set: \t\t{}".format(test_x.shape))

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
#test_data = TensorDataset(torch.from_numpy(test_title))

# dataloaders
batch_size = 20

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
#test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
dataiter = iter(valid_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)

print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

In [ ]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

In [ ]:
import torch.nn as nn

class Q_RNN(nn.Module):
    """
    The RNN model that will be used to find insincere questions
    """

    def __init__(self, vocab_size, output_size, embedding_dim,out_channels, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv = nn.Conv1d(embedding_dim,hidden_dim,1)
        self.p1 = nn.AvgPool1d(1)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
        #print(embeds.size())
        embeds = embeds.transpose(0, 1).transpose(1, 2)
        #print(embeds.size())
        embeds = self.conv(embeds)
        #print(embeds.size())
        embeds = self.p1(embeds)
        #print(embeds.size())
        embeds = embeds.transpose(1, 2).transpose(0, 1)
        #print(embeds.size())
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
q_size = len(q_vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 10
conv_size = 20
hidden_dim = 64
n_layers = 2

net = SentimentRNN(q_size, output_size, embedding_dim,conv_size, hidden_dim, n_layers)

print(net)

In [ ]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
epochs = 3 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            #val_h = net.init_hidden(batch_size)
            #val_losses = []
            #net.eval()
            #for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                #val_h = tuple([each.data for each in val_h])

                #if(train_on_gpu):
                #    inputs, labels = inputs.cuda(), labels.cuda()

               # output, val_h = net(inputs, val_h)
              #  val_loss = criterion(output.squeeze(), labels.float())

             #   val_losses.append(val_loss.item())

            #net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()))
                  #"Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
test_data_path = '../input/test.csv'

# read test data file using pandas
test_df = pd.read_csv(test_data_path)

In [ ]:
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
sample.head()

In [ ]:
test_df.head()

In [ ]:
test_text = test_df['question_text'].str.replace('[^\w\s]','')

In [ ]:
test_text = test_text.str.split(' ')

In [ ]:

test_int=[]
for tag in test_text:
  if isinstance(tag,list): 
      if len(tag)!=0:
        try:
          test_int.append([q_vocab_to_int[word] for word in tag])
        except KeyError as error:
          test_int.append([0]) 
for i in range(10):
    test_int.append([0])

In [ ]:
seq_length = 25

test_features = pad_features(test_int, seq_length=seq_length)

## test statements - do not change - ##
assert len(test_features)==len(test_int), "Your features should have as many rows as reviews."
assert len(test_features[0])==seq_length, "Each feature row should contain seq_length values."
print(test_features[:30,:25])

In [ ]:
len(test_features)

In [ ]:

#np.append(test_features,m)

In [ ]:
test_data = TensorDataset(torch.from_numpy(test_features))

In [ ]:
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [ ]:
op = []
for inputs in test_loader:
    inputs = inputs[0]
    inputs = inputs.cuda()
    x,h = net(inputs,h)
    for i in x:
        if i<0.5:
            op.append(0)
        else:
            op.append(1)
            

In [ ]:
op

In [ ]:
qid = test_df['qid']

In [ ]:
output = pd.DataFrame({'qid': qid,
                       'prediction': op[:-10]})

In [ ]:
output.to_csv('submission.csv', index=False)